### Тема “POS-tagger и NER”

#### Задание 1. 
Написать теггер на данных с русским языком
- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
- сравнить все реализованные методы сделать выводы
 


In [45]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, PER
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from razdel import tokenize

import warnings
warnings.filterwarnings("ignore")

In [6]:
!wget http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip

--2023-05-19 20:25:20--  http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip
Resolving ai-center.botik.ru (ai-center.botik.ru)... 95.129.138.2
Connecting to ai-center.botik.ru (ai-center.botik.ru)|95.129.138.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3363777 (3.2M) [application/zip]
Saving to: 'Persons-1000.zip'

Persons-1000.zip    100%[===================>]   3.21M   217KB/s    in 17s     

2023-05-19 20:25:38 (197 KB/s) - 'Persons-1000.zip' saved [3363777/3363777]



In [9]:
# nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /Users/aleksandrkrylov/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.


True

In [7]:
from corus import load_persons

dir = 'Persons-1000.zip'
records = load_persons(dir)
next(records)

PersonsMarkup(
    text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
    spans=[PersonsSpan(
         id=1,
         start=308,
         stop=324,
         value='ГРИГОРИЙ КАР

In [12]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [13]:
data = []
for record in records:
    doc = Doc(record.text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        data.append((token.text, token.pos))

In [15]:
data[:10]

[('4', 'ADJ'),
 ('октября', 'NOUN'),
 ('назначены', 'VERB'),
 ('очередные', 'ADJ'),
 ('выборы', 'NOUN'),
 ('Верховного', 'ADJ'),
 ('Совета', 'PROPN'),
 ('Аджарской', 'PROPN'),
 ('АР', 'PROPN'),
 ('По', 'ADP')]

In [24]:
X, y = zip(*data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [17]:
unigram_tagger = UnigramTagger([list(zip(X_train, y_train))])
unigram_acc = unigram_tagger.evaluate([list(zip(X_test, y_test))])

bigram_tagger = BigramTagger([list(zip(X_train, y_train))])
bigram_acc = bigram_tagger.evaluate([list(zip(X_test, y_test))])

trigram_tagger = TrigramTagger([list(zip(X_train, y_train))])
trigram_acc = trigram_tagger.evaluate([list(zip(X_test, y_test))])

bigram_tagger = BigramTagger([list(zip(X_train, y_train))], backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate([list(zip(X_test, y_test))])

trigram_tagger = TrigramTagger([list(zip(X_train, y_train))], backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate([list(zip(X_test, y_test))])

print(f'Accuracy:\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 5)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n')

Accuracy:
Unigram Tagger: 0.9,
Bigram Tagger: 0.0,
Trigram Tagger: 0.0,
Bigram and Unigram Tagger: 0.89811,
Trigram, Bigram and Unigram Tagger: 0.8975,



In [18]:
# Custom tagger

In [25]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [26]:
vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train_vec, y_train_encoded)
    pred = lr.predict(X_test_vec)
    f1 = f1_score(y_test_encoded, pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(y_test_encoded, pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(y_test_encoded, pred, target_names=le.classes_))

CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.94      0.94      0.94      5127
         ADP       0.97      1.00      0.98      5474
         ADV       0.97      0.93      0.95      1304
         AUX       0.95      1.00      0.97       456
       CCONJ       0.88      0.99      0.93       996
         DET       0.83      0.84      0.84       793
        INTJ       0.00      0.00      0.00         1
        NOUN       0.95      0.97      0.96     14653
         NUM       0.80      0.84      0.82       602
        PART       0.96      0.83      0.89       563
        PRON       0.86      0.88      0.87      1365
       PROPN       0.92      0.85      0.89      6656
       PUNCT       1.00      1.00      1.00      8698
       SCONJ       0.93      0.98      0.95       732
         SYM       1.00      0.98      0.99        49
        VERB       0.97      0.96      0.97      5170
           X       0.86     

In [27]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.949553
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.942610
8,"HashingVectorizer(analyzer='char', n_features=...",0.923656
7,"HashingVectorizer(analyzer='char', n_features=...",0.920570
6,"HashingVectorizer(analyzer='char', n_features=...",0.914419
2,"HashingVectorizer(analyzer='char', n_features=...",0.898793
3,"CountVectorizer(ngram_range=(1, 3))",0.752360
4,"TfidfVectorizer(ngram_range=(1, 3))",0.745504
11,"HashingVectorizer(n_features=5000, ngram_range...",0.680992
10,"HashingVectorizer(n_features=3000, ngram_range...",0.656369


In [28]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.949941
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.943141
8,"HashingVectorizer(analyzer='char', n_features=...",0.924837
7,"HashingVectorizer(analyzer='char', n_features=...",0.921682
6,"HashingVectorizer(analyzer='char', n_features=...",0.915713
2,"HashingVectorizer(analyzer='char', n_features=...",0.900298
3,"CountVectorizer(ngram_range=(1, 3))",0.734633
4,"TfidfVectorizer(ngram_range=(1, 3))",0.727568
11,"HashingVectorizer(n_features=5000, ngram_range...",0.683290
10,"HashingVectorizer(n_features=3000, ngram_range...",0.664117


### Задание 2. 
- Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
- проверить NER из nltk/spacy/deeppavlov
- написать свой нер попробовать разные подходы
  - передаём в сетку токен и его соседей
  - передаём в сетку только токен
  - свой вариант
- сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)

In [29]:
# !wget http://www.labinform.ru/pub/named_entities/collection5.zip
# !unzip collection5.zip

--2023-05-20 11:54:45--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: 'collection5.zip'

collection5.zip     100%[===================>]   1.81M  1008KB/s    in 1.8s    

2023-05-20 11:54:48 (1008 KB/s) - 'collection5.zip' saved [1899530/1899530]

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection

In [31]:
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [32]:
from corus import load_ne5
records = load_ne5('Collection5/')
document = next(records).text
document

'Жириновский предлагает обменять с США Сноудена на Бута\r\n\r\nЛидер ЛДПР Владимир Жириновский предложил обменять бывшего сотрудника ЦРУ США Эдварда Сноудена, который прибыл в Москву, на осужденного в Америке бизнесмена Виктора Бута.\r\n\r\n"Сноудена ни в коем случае не высылать в США, а обменять на Виктора Бута и Константина Ярошенко. В идеале — добавить генерала Олега Калугина", — написал он в своем микроблоге в Twitter.\r\n\r\nСноуден, работавший на компанию Booz Allen Hamilton — подрядчика Центрального разведывательного управления США, в начале июня распространил секретный ордер суда, по которому спецслужбы получили доступ ко всем звонкам крупнейшего сотового оператора Verizon, а также данные о сверхсекретной программе агентства национальной безопасности PRISM, позволяющей отслеживать электронные коммуникации на крупнейших сайтах. В воскресенье стало известно, что Сноуден прибыл из Гонконга в Москву и запросил убежища в Эквадоре.\r\n\r\nЧто ждет Эдварда Сноудена\r\n\r\nЭдвард Сноуд

In [33]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Hamilton', 'PERSON'),
 ('Америке', 'PERSON'),
 ('Виктора Бута', 'PERSON'),
 ('Москву', 'PERSON'),
 ('Сноуден', 'PERSON'),
 ('Эдварда Сноудена', 'PERSON'),
 ('Эдварда Сноудена Эдвард Сноуден', 'PERSON')}

In [35]:
# !pip install spacy

In [36]:
# !python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 9.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 955.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 6.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [39]:
import spacy
from spacy import displacy

nlp = spacy.load('ru_core_news_sm')

ny_bb = document
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [40]:
#список токенов, частей речи и сущностей
for token in article:
    print(token.text, token.pos_, token.dep_)

Жириновский PROPN nsubj
предлагает VERB ROOT
обменять VERB xcomp
с ADP case
США PROPN obl
Сноудена PROPN obj
на ADP case
Бута PROPN obl


 SPACE dep
Лидер NOUN nsubj
ЛДПР PROPN nmod
Владимир PROPN appos
Жириновский PROPN flat:name
предложил VERB conj
обменять VERB xcomp
бывшего ADJ amod
сотрудника NOUN obj
ЦРУ PROPN nmod
США PROPN nmod
Эдварда PROPN appos
Сноудена PROPN flat:name
, PUNCT punct
который PRON nsubj
прибыл VERB acl:relcl
в ADP case
Москву PROPN obl
, PUNCT punct
на ADP case
осужденного VERB acl
в ADP case
Америке PROPN obl
бизнесмена NOUN conj
Виктора PROPN appos
Бута PROPN flat:name
. PUNCT punct


 SPACE dep
" PUNCT punct
Сноудена NOUN ROOT
ни PART cc
в ADP fixed
коем DET fixed
случае NOUN fixed
не PART advmod
высылать VERB xcomp
в ADP case
США PROPN obl
, PUNCT punct
а CCONJ cc
обменять VERB conj
на ADP case
Виктора PROPN obl
Бута PROPN flat:name
и CCONJ cc
Константина PROPN conj
Ярошенко PROPN flat:name
. PUNCT punct
В ADP case
идеале NOUN ROOT
— PUNCT punct
добавить V

### NLTK

In [46]:
import nltk

In [47]:
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/aleksandrkrylov/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/aleksandrkrylov/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [48]:
for ix, rec in enumerate(records):
  print(rec.text)
  print('\nИменованные сущности:')
  for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(rec.text), lang='rus')):
    if hasattr(chunk, 'label'):
      print(f'{chunk} - {chunk.label()}')
  break

Д.Медведев назначил ряд глав региональных МВД

Президент России Дмитрий Медведев сегодня своим указом назначил на должности глав девяти региональных управлений и Министерств внутренних дел (УВД и МВД), сообщает пресс-служба главы государства.

Генерал-майор полиции Михаил Закомалдин назначен главой МВД по Республике Карелия, генерал-майор полиции Виктор Кошелев занял должность начальника МВД по Республике Саха (Якутия), полковник полиции Александр Речицкий возглавил МВД по Республике Адыгея, полковник полиции Сергей Семенов стал министром внутренних дел по Чувашской Республике, полковник полиции Александр Удовенко возглавил МВД по Республике Алтай.

Кроме того, начальником управления МВД РФ по Ханты-Мансийскому автономному округу - Югре стал генерал-майор полиции Василий Романиц, начальником главного управления МВД РФ по Новосибирской области назначен генерал-лейтенант полиции Сергей Глушков, должность начальника управления МВД РФ по Ульяновской области занял полковник полиции Андрей Л

#### Natasha

In [49]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)

# Инициализация компонентов Natasha
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

for record in records:
    text = record.text
    doc = Doc(text)

    # Разбивка текста на предложения
    doc.segment(segmenter)

    # Применение модели NER
    doc.tag_ner(ner_tagger)

    # Последующая обработка для улучшения качества имен и фамилий
    for span in doc.spans:
        span.normalize(morph_vocab)

    for span in doc.spans:
        if span.type == PER:
            print(f'Name: {span.normal}')


Name: В.Суркову
Name: Владислава Суркова
Name: В.Суркову
Name: В.Сурков
Name: Владимира Путина
Name: Вячеслава Володина
Name: В.Путина
Name: В.Володина
Name: Дмитрий Медведев
Name: Хуршед Нуманов
Name: Михаил Слизков
Name: Александр Артемьев
Name: Геннадий Барковский
Name: Владимир Иоголевич
Name: Валерий Назаров
Name: Григорий Скрипкин
Name: Юрия Бородастова
Name: Фаима Мухаметшина
Name: Леонида Хейло
Name: Д.Медведев
Name: Дмитрий Медведев
Name: Вячеслав Хаустов
Name: В.Хаустова
Name: Николай Войтенков
Name: Александр Макаров
Name: Сергея Козлова
Name: Сергея Коробова
Name: Сергей Солодовников
Name: Евгений Соловьев
Name: Андрей Юдинцев
Name: Сергеем Нарышкиным
Name: Богуслаев
Name: Виктор Авдеенко
Name: Вячеслав Богуслаев
Name: Богуслаеву Вячеславу Александровичу
Name: Як-130
Name: Ю.Балуевского
Name: Дмитрий Медведев
Name: Юрия Балуевского
Name: Ю.Балуевскому
Name: Ю.Балуевский
Name: Ю.Балуевский
Name: Н.Макаров
Name: Юрий Балуевский
Name: Рогозина
Name: Дмитрия Рогозина
Name: Алек

#### NN

In [ ]:
#!pip install corus

In [52]:
dir = 'Collection5/'
records = load_ne5(dir)

In [53]:
from razdel import tokenize

words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [54]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words.head()

,word,tag
0,Жириновский,PER
1,предлагает,OUT
2,обменять,OUT
3,с,OUT
4,США,GEOPOLIT


In [55]:
df_words['tag'].value_counts()

OUT         219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [71]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [57]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
train_x.apply(len).max(axis=0)

55

In [58]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [59]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [66]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

# без соседних токенов 
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len)


# с соседними токенами
vectorize_layer_n13 = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=(1, 3),
    output_sequence_length=seq_len)

vectorize_layer_n4 = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=4,
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [61]:
len(vectorize_layer.get_vocabulary())

29912

In [72]:
embedding_dim = 128

modeln = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim),
    Bidirectional(GRU(64)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

In [73]:
modeln.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [74]:
modeln.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12444/12444 [==============================] - 158s 13ms/step - loss: 0.3061 - accuracy: 0.9128 - val_loss: 0.2139 - val_accuracy: 0.9391
Epoch 2/3
12444/12444 [==============================] - 163s 13ms/step - loss: 0.1386 - accuracy: 0.9611 - val_loss: 0.2202 - val_accuracy: 0.9418
Epoch 3/3
12444/12444 [==============================] - 166s 13ms/step - loss: 0.1204 - accuracy: 0.9643 - val_loss: 0.2288 - val_accuracy: 0.9418


In [75]:
vectorize_layer_n13.adapt(text_data)

modeln13 = Sequential([
    vectorize_layer_n13,
    Embedding(vocab_size, embedding_dim),
    Bidirectional(GRU(64)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

modeln13.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

modeln13.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12444/12444 [==============================] - 204s 16ms/step - loss: 0.3041 - accuracy: 0.9142 - val_loss: 0.2141 - val_accuracy: 0.9393
Epoch 2/3
12444/12444 [==============================] - 198s 16ms/step - loss: 0.1393 - accuracy: 0.9610 - val_loss: 0.2529 - val_accuracy: 0.8935
Epoch 3/3
12444/12444 [==============================] - 211s 17ms/step - loss: 0.1210 - accuracy: 0.9644 - val_loss: 0.2976 - val_accuracy: 0.8936


In [76]:
vectorize_layer_n4.adapt(text_data)

modeln4 = Sequential([
    vectorize_layer_n4,
    Embedding(vocab_size, embedding_dim),
    Bidirectional(GRU(64)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

modeln4.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

modeln4.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12444/12444 [==============================] - 232s 18ms/step - loss: 0.3062 - accuracy: 0.9139 - val_loss: 0.2158 - val_accuracy: 0.9398
Epoch 2/3
12444/12444 [==============================] - 245s 20ms/step - loss: 0.1393 - accuracy: 0.9605 - val_loss: 0.2805 - val_accuracy: 0.8935
Epoch 3/3
12444/12444 [==============================] - 189s 15ms/step - loss: 0.1214 - accuracy: 0.9641 - val_loss: 0.2457 - val_accuracy: 0.8945


#### Мы видим что результаты моделей с группами токенов показали результаты хуже с выбранной архитектурой сети, причем результаты похожи на переобучение несмотря на то что используется дропаут слой. Проверим более расширенные метрики по классам

In [77]:
labels_predict_n = modeln.predict(valid_data)
labels_predict_n13 = modeln13.predict(valid_data)
labels_predict_n4 = modeln4.predict(valid_data)

4148/4148 [==============================] - 5s 1ms/step


In [78]:
class_preds_n = np.argmax(tf.nn.softmax(labels_predict_n), axis=1)
class_preds_n13 = np.argmax(tf.nn.softmax(labels_predict_n13), axis=1)
class_preds_n4 = np.argmax(tf.nn.softmax(labels_predict_n4), axis=1)

In [79]:
valid_y = encoder.inverse_transform(valid_y)
class_preds_n = encoder.inverse_transform(class_preds_n)
class_preds_n13 = encoder.inverse_transform(class_preds_n13)
class_preds_n4 = encoder.inverse_transform(class_preds_n4)

In [80]:
print(classification_report(valid_y, class_preds_n))

              precision    recall  f1-score   support

    GEOPOLIT       0.91      0.90      0.90      1149
         LOC       0.85      0.81      0.83      1148
       MEDIA       0.92      0.79      0.85       638
         ORG       0.87      0.55      0.68      3344
         OUT       0.94      0.99      0.97     54781
         PER       0.99      0.70      0.82      5308

    accuracy                           0.94     66368
   macro avg       0.91      0.79      0.84     66368
weighted avg       0.94      0.94      0.94     66368



In [81]:
print(classification_report(valid_y, class_preds_n13))

              precision    recall  f1-score   support

    GEOPOLIT       0.90      0.89      0.90      1149
         LOC       0.84      0.82      0.83      1148
       MEDIA       0.93      0.79      0.85       638
         ORG       0.86      0.56      0.68      3344
         OUT       0.97      0.92      0.94     54781
         PER       0.49      0.87      0.63      5308

    accuracy                           0.89     66368
   macro avg       0.83      0.81      0.80     66368
weighted avg       0.92      0.89      0.90     66368



In [82]:
print(classification_report(valid_y, class_preds_n4))

              precision    recall  f1-score   support

    GEOPOLIT       0.92      0.89      0.91      1149
         LOC       0.85      0.82      0.83      1148
       MEDIA       0.92      0.78      0.85       638
         ORG       0.88      0.55      0.68      3344
         OUT       0.97      0.92      0.94     54781
         PER       0.50      0.86      0.63      5308

    accuracy                           0.89     66368
   macro avg       0.84      0.81      0.81     66368
weighted avg       0.92      0.89      0.90     66368



Видим что при использовании нграмм токенов результаты ухудшаются значительнее всего на классе Person, вероятно по причине того что данным именованная сущность мало зависит от контекта